In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Download the dataset and unzip it
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2024-05-07 19:08:24--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2024-05-07 19:08:24--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  24.3MB/s    in 10s     

2024-05-07 19:08:34 (23.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
!unzip -qq 'tiny-imagenet-200.zip'
!ls

drive  sample_data  tiny-imagenet-200  tiny-imagenet-200.zip


In [ ]:
# Input image dimensions
img_height, img_width, channels = 64, 64, 3

# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 24
num_train = 100000
num_validation = 10000

# Data generators
train_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
    './tiny-imagenet-200/train/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 100000 images belonging to 200 classes.


In [ ]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None,
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(
    val_data, directory='./tiny-imagenet-200/val/images/',
    x_col='File', y_col='Class',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False)



Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
# Model
input = Input(shape=(img_height, img_width, channels))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.regularizers import l2

# Creating the sequential model
model = Sequential([
    # First convolutional block
    Conv2D(32, (3, 3), padding='same', kernel_initializer="he_normal", kernel_regularizer=l2(1e-4), input_shape=(img_height, img_width, channels)),
    BatchNormalization(),
    Activation('relu'),

    # Second convolutional block
    Conv2D(128, (3, 3), padding='same', kernel_initializer="he_normal", kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Third convolutional block
    Conv2D(256, (3, 3), padding='same', kernel_initializer="he_normal", kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Fourth convolutional block
    Conv2D(512, (3, 3), padding='same', kernel_initializer="he_normal", kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Classification layer
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
])

# Model Summary
model.summary()

# Compile the Model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       36992     
                                                                 
 batch_normalization_1 (Bat  (None, 64, 64, 128)       512       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 64, 64, 128)       0

In [ ]:
# Callbacks
checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/model.hdf5", verbose=1, save_best_only=True)

# Fit the Model
model.fit(train_generator,
          epochs=epochs,
          steps_per_epoch=num_train // batch_size,
          validation_data=validation_generator,
          validation_steps=num_validation // batch_size,
          callbacks=[checkpointer],
          verbose=1)

Epoch 1/24
781/781 [==============================] - ETA: 0s - loss: 4.4057 - accuracy: 0.1144
Epoch 1: val_loss improved from inf to 4.12119, saving model to /content/drive/My Drive/Colab Notebooks/model.hdf5
781/781 [==============================] - 122s 142ms/step - loss: 4.4057 - accuracy: 0.1144 - val_loss: 4.1212 - val_accuracy: 0.1429


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/24
781/781 [==============================] - ETA: 0s - loss: 3.7606 - accuracy: 0.2018
Epoch 2: val_loss did not improve from 4.12119
781/781 [==============================] - 105s 135ms/step - loss: 3.7606 - accuracy: 0.2018 - val_loss: 4.2685 - val_accuracy: 0.1538
Epoch 3/24
781/781 [==============================] - ETA: 0s - loss: 3.4666 - accuracy: 0.2548
Epoch 3: val_loss improved from 4.12119 to 3.92413, saving model to /content/drive/My Drive/Colab Notebooks/model.hdf5
781/781 [==============================] - 108s 138ms/step - loss: 3.4666 - accuracy: 0.2548 - val_loss: 3.9241 - val_accuracy: 0.1879
Epoch 4/24
781/781 [==============================] - ETA: 0s - loss: 3.2760 - accuracy: 0.2929
Epoch 4: val_loss did not improve from 3.92413
781/781 [==============================] - 108s 138ms/step - loss: 3.2760 - accuracy: 0.2929 - val_loss: 4.0803 - val_accuracy: 0.1928
Epoch 5/24
781/781 [==============================] - ETA: 0s - loss: 3.1286 - accuracy: 0.321